<a href="https://colab.research.google.com/github/MohitD017/Colab_RAG_CyberForensics/blob/main/Colab_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step-1: Install the Necessary Libraries

!pip install transformers langchain langchain_community faiss-cpu huggingface_hub pypdf
!pip install pymupdf
!pip install -U langchain langchain-huggingface
!pip install --upgrade langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Step-2: Import the Required Modules from the Installed Libraries

import os
import requests
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings, HuggingFaceHubEmbeddings
from langchain.llms import HuggingFaceEndpoint
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import textwrap

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Or suppress specific warnings (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)

import logging

# Suppress all log messages below the ERROR level
logging.getLogger("langchain_community").setLevel(logging.ERROR)

In [ ]:
# Step-3: Checking if Langchain is Tnstalled Properly

!pip show langchain

Name: langchain
Version: 0.3.20
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [ ]:
# Step-4: Provide the Unique Knowledge Base of Your Choice
document_text = """
The neon lights of X shimmered, reflecting off the sleek cybernetic implants of its citizens. Detective Y, however, saw little of the city's beauty as he hunched over a holographic display, a frown etched on his face. He was facing a digital enigma: a ransomware attack unlike any he'd encountered before. The victim, a renowned robotics engineer named Z, reported that all his research data, years of work on a groundbreaking AI-powered prosthetic limb, had been encrypted. The perpetrator, a shadowy entity calling themselves The Serpent, demanded an exorbitant ransom in untraceable cryptocurrency. Y, a veteran of the Cyber Crimes Division, knew that time was of the essence. Z's research was not only invaluable scientifically but also held the potential to revolutionize prosthetics for millions. But the initial investigation yielded little. The Serpent had left no digital footprints, employing advanced encryption and anonymization techniques to mask their identity and location. Y, however, was not one to be easily deterred. He understood the power of expanding the knowledge base. He requested and received access to Z's entire digital life – his personal computers, lab servers, cloud storage, even his smart home devices. Y's team, equipped with cutting-edge forensic tools, began their meticulous analysis. They reconstructed deleted files, analyzed network traffic logs, and even delved into the firmware of Z's smart appliances, searching for any hidden data or unusual connections. They expanded their search beyond Z's immediate digital sphere, examining online forums, academic databases, and even dark web marketplaces for any mention of the stolen research or clues about The Serpent's identity. As the team dug deeper, they discovered a seemingly unrelated incident: a minor security breach at a local university's robotics lab a few weeks prior. The breach, initially dismissed as a student prank, involved the theft of a small, experimental AI algorithm. Y's intuition flared. Could this be connected to The Serpent's attack? Further investigation revealed a startling connection. The stolen algorithm, while seemingly insignificant on its own, was a crucial component in Z's research. The Serpent, it seemed, had planned their attack meticulously, acquiring the necessary tools before launching their ransomware scheme. With this expanded knowledge base, Y's team was able to trace The Serpent's digital trail. They uncovered a hidden connection to a remote server located in the abandoned industrial sector of X. A raid on the location led to the arrest of a disgruntled former student of Z's, seeking revenge for a perceived academic slight. The case of The Serpent highlighted the crucial role of expanding the knowledge base in digital forensics. By connecting seemingly disparate pieces of information and exploring every digital avenue, Y and his team were able to bring a cybercriminal to justice and safeguard groundbreaking research that held the promise of a better future.
"""

# Step-5: Split and Visualize the Text in Chunks

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=25,  # Optional: Add overlap for context
    separators=["\n\n", "\n", ". ", " ", ""],  # Define splitting hierarchy
    length_function=len  # Use character count as the length metric
)

def para2Chunkc(paragraph):
    """Function to split a paragraph into chunks."""
    # Ensure the input is a string
    if not isinstance(paragraph, str):
        raise TypeError("Input must be a string.")
    chunks = splitter.split_text(paragraph)
    return chunks

# Print the original paragraph and its corresponding chunks
print("Knowledge Base:")
KnowlwdgeBase = textwrap.fill(document_text, width=165)  # Wrap text to 165 characters per line
print("\n \t", KnowlwdgeBase)


# Split the document text into chunks
print("\nChunks and their lengths:")
chunks = para2Chunkc(document_text)

# Print each chunk along with its length
for i, chunk in enumerate(chunks):
    print(f"\nChunk {i+1} (Length: {len(chunk)} characters):")
    print("--------------------------------")
    chunk = textwrap.fill(chunk, width=165)  # Wrap each chunk to 165 characters per line
    print("\t", chunk)

Knowledge Base:

 	  The neon lights of X shimmered, reflecting off the sleek cybernetic implants of its citizens. Detective Y, however, saw little of the city's beauty as he hunched
over a holographic display, a frown etched on his face. He was facing a digital enigma: a ransomware attack unlike any he'd encountered before. The victim, a
renowned robotics engineer named Z, reported that all his research data, years of work on a groundbreaking AI-powered prosthetic limb, had been encrypted. The
perpetrator, a shadowy entity calling themselves The Serpent, demanded an exorbitant ransom in untraceable cryptocurrency. Y, a veteran of the Cyber Crimes Division,
knew that time was of the essence. Z's research was not only invaluable scientifically but also held the potential to revolutionize prosthetics for millions. But the
initial investigation yielded little. The Serpent had left no digital footprints, employing advanced encryption and anonymization techniques to mask their identity
and 

In [ ]:
# Step-6: Setting-up the HuggingFace Inference API Embedding

HF_API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/paraphrase-MiniLM-L6-v2"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'ENTER YOUR HUGGINGFACE API TOKEN'

headers = {
    "Authorization": f"Bearer {os.getenv('HUGGINGFACEHUB_API_TOKEN')}"
}

def get_embeddings(texts):
    """Function to get embeddings using Hugging Face API"""
    payload = {
        "inputs": texts,
        "options": {"wait_for_model": True}
    }
    response = requests.post(HF_API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        embeddings = response.json()
        return embeddings
    else:
        print(f"Request failed with status code {response.status_code}")
        print(f"Error: {response.text}")
        return

In [ ]:
# Step-7: Vectorisation of the Embedded Text

embeddings = get_embeddings(chunks)  # Get embeddings for document chunks

# Initialize text_embeddings_dict with default values
text_embeddings_dict = {
    "texts": [],
    "embeddings": []
}

if embeddings:
    text_embeddings_dict = {
        "texts": chunks,
        "embeddings": embeddings
    }

    # Print the full API response and truncated embeddings
    print(embeddings)  # Pretty-print the raw response
    print("\nTruncated Embeddings:")
    print("---------------------------------------------------------------")
    for i, embedding in enumerate(embeddings[:1]):
        print(f"\t Text Embedding for Chunk {i+1}: {embedding[:10]}... [remaining elements truncated]")

    def embedding_functions(texts):
        """Generate embeddings for new queries using the Hugging Face API."""
        return get_embeddings(texts)

    vectorstore = FAISS.from_embeddings(
        text_embeddings=list(zip(text_embeddings_dict["texts"], text_embeddings_dict["embeddings"])),
        embedding=embedding_functions
    )

    # --- Show One Example of the Chunk and Its Corresponding Embedding ---
    if len(chunks) > 0:
        example_chunk = chunks[0]
        example_embedding = embeddings[0]

        print("\nExample Chunk and Its Corresponding Embedding:")
        print("--------------------------------------------------------")
        print(f"\n \t Text Chunk 1: {example_chunk}")
        print(f"\t Text Embedding of Chunk 1: {example_embedding}")

[[-0.3275904655456543, 0.39002105593681335, -0.25993314385414124, -0.2682356536388397, 0.20118968188762665, -0.21115125715732574, 0.45387008786201477, -0.10768695920705795, -0.06840736418962479, -0.033133283257484436, 0.21782395243644714, 0.199327751994133, 0.31692370772361755, -0.18796208500862122, -0.043666914105415344, -0.10008681565523148, -0.08619167655706406, -0.34208759665489197, 0.20552712678909302, 0.04054751992225647, 0.20095258951187134, -0.4266882836818695, 0.2502999007701874, -0.4903154969215393, 0.1595451831817627, 0.08141636848449707, 0.29145947098731995, -0.07534435391426086, -0.1643414944410324, 0.03158436715602875, 0.43301278352737427, -0.09865471720695496, 0.03912290558218956, 0.04644562676548958, 0.01497711706906557, -0.10925950109958649, 0.306766152381897, 0.0655030831694603, -0.17317336797714233, -0.2342679500579834, -0.23407647013664246, 0.34936797618865967, 0.012806370854377747, 0.23215998709201813, 0.17119768261909485, 0.19519154727458954, 0.10998321324586868, 

In [ ]:
# Step-8: Creating the RetrievalQA Chain for Answering the Questions
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/microsoft/phi-2",  # Updated Model
    task="text-generation",
    temperature=0.1
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3}),
    chain_type="stuff",
)

# --- Print Stored Vectors and Their Structure ---
print("\nStored Vectors and Their Structure:")
print("----------------------------------------")
for i, (text, embedding) in enumerate(zip(text_embeddings_dict["texts"], text_embeddings_dict["embeddings"])):
    print(f"\n \tText Chunk {i + 1}: {text}")
    print(f"\tEmbedding {i + 1}: {embedding}")
    print(f"\tEmbedding Type: {type(embedding)}")
    print(f"\tEmbedding Shape: {len(embedding)}")

# --- Print QA Chain Configuration ---
print("\nQA Chain Configuration")
print("----------------------")
print(f"\tllm Model: {llm.endpoint_url}")
print(f"\tllm Task: {llm.task}")
print(f"\tllm Temperature: {llm.temperature}")
print(f"\tRetriever Search Type: {qa_chain.retriever.search_type}")
print(f"\tRetriever Top-K: {qa_chain.retriever.search_kwargs.get('k', 'N/A')}")


Stored Vectors and Their Structure:
----------------------------------------

 	Text Chunk 1: The neon lights of X shimmered, reflecting off the sleek cybernetic implants of its citizens. Detective Y, however, saw little of the city's beauty as he hunched over a holographic display, a frown etched on his face. He was facing a digital enigma: a ransomware attack unlike any he'd encountered before
	Embedding 1: [-0.3275904655456543, 0.39002105593681335, -0.25993314385414124, -0.2682356536388397, 0.20118968188762665, -0.21115125715732574, 0.45387008786201477, -0.10768695920705795, -0.06840736418962479, -0.033133283257484436, 0.21782395243644714, 0.199327751994133, 0.31692370772361755, -0.18796208500862122, -0.043666914105415344, -0.10008681565523148, -0.08619167655706406, -0.34208759665489197, 0.20552712678909302, 0.04054751992225647, 0.20095258951187134, -0.4266882836818695, 0.2502999007701874, -0.4903154969215393, 0.1595451831817627, 0.08141636848449707, 0.29145947098731995, -0.0753443

In [ ]:
# Step-8: Process all chunks to generate all embeddings for all chunks
def processAllText(chunks):
    """Generate embeddings for all text chunks and print them."""
    # Get embeddings for all chunks
    embeddings = get_embeddings(chunks)

    print(f"\n Generating all the Chunks with the Corresponding Embeds:")
    print("--------------------------------------------------------")
    if embeddings:
        # Print each chunk along with its embedding
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            # Wrap the text chunk for better readability
            wrapped_chunk = textwrap.fill(chunk, width=165)  # Adjust width as needed
            print(f"\n Chunk {i + 1} (Length: {len(chunk)} characters):")
            print("--------------------------------")
            print(f"\t Text Chunk:\n{wrapped_chunk}")  # Print wrapped text
            print(f"\t Embedding: {embedding[:10]}... [remaining elements truncated]")
            print(f"\t Embedding Type: {type(embedding)}")
            print(f"\t Embedding Shape: {len(embedding)}")

        return embeddings
    else:
        print("❌ No embeddings were generated. Please check the error in API response.")
        return []

# Example usage
if __name__ == "__main__":
    # Assuming 'chunks' is already defined from previous steps
    all_embeddings = processAllText(chunks)


 Generating all the Chunks with the Corresponding Embeds:
--------------------------------------------------------

 Chunk 1 (Length: 304 characters):
--------------------------------
	 Text Chunk:
The neon lights of X shimmered, reflecting off the sleek cybernetic implants of its citizens. Detective Y, however, saw little of the city's beauty as he hunched
over a holographic display, a frown etched on his face. He was facing a digital enigma: a ransomware attack unlike any he'd encountered before
	 Embedding: [-0.3275904655456543, 0.39002105593681335, -0.25993314385414124, -0.2682356536388397, 0.20118968188762665, -0.21115125715732574, 0.45387008786201477, -0.10768695920705795, -0.06840736418962479, -0.033133283257484436]... [remaining elements truncated]
	 Embedding Type: <class 'list'>
	 Embedding Shape: 384

 Chunk 2 (Length: 374 characters):
--------------------------------
	 Text Chunk:
. The victim, a renowned robotics engineer named Z, reported that all his research data, year

In [ ]:
# Step-9: Establishing a Chat Interface

print("\n||🤖 Your RAG Model is Ready! Type 'exit' to quit.||\n")
while True:
    query = input("🟢 Your Question: ")
    if query.lower() == "exit":
        print("\n👋 Goodbye!")
        break
    try:
        response = qa_chain.invoke(query)
        print(f"\n🔵 RAG's Answer: {response['result']}\n")
    except Exception as e:
        print(f"\n❌ **Error:** {e}")
else:
    print("❌ No embeddings were generated. Please check the error in API response.")


||🤖 Your RAG Model is Ready! Type 'exit' to quit.||

🟢 Your Question: What was the victim's profession?

🔵 RAG's Answer:  The victim's profession was a robotics engineer.

🟢 Your Question: What was the name of the university where the minor security breach occurred?

🔵 RAG's Answer:  The university was not named in the provided context.

🟢 Your Question: exit

👋 Goodbye!


# **TEST QUESTIONS:**

## In-Text Questions:

1. What type of cyberattack did Detective Y investigate? (Answer: Ransomware attack)
2. What was the victim's profession? (Answer: Robotics engineer)
3. Where was the remote server located that ultimately led to the perpetrator's arrest? (Answer: Abandoned industrial sector of city X)


## Out-of-Text Questions:
1. What specific encryption algorithm did The Serpent use to encrypt the research data? (The story doesn't mention the specific algorithm.)
2. What was the name of the university where the minor security breach occurred? (The story doesn't mention the university's name.)
3. Did Detective Y's team collaborate with any external cybersecurity experts or organizations during the investigation? (The story doesn't mention any external collaboration.)